# penjelaskan Naive Bayes

Algoritma Naive Bayes (NB) adalah metode yang menggunakan prinsip probability utk membuat model prediksi klasifikasi.

Keunggulan NB adalah sifatnya yg efektif dan cepat utk mengolalh data berjumblah besar.

## Bagaimana Naive Bayes Bekerja

NB adalah sekumpulan (bukan hanya satu) algoritma klasifikasi yg dibangun berdasarkan Teori Bayes.

Model klasifikasi yg memakai metode bayes memanfaatkan training dataset utk menghitung kemungkinan setiap kelas berdasarkan nilai-nilai feature di dalamnya.

conditional probability yaitu kemungkinan munculnya suatu kejadian A bila suatu kejadian B muncul. Karena kejadian A bergantung kpd kejadian B, itu sebabnya disebut conditional(bersyarat).